In [2]:
# Requires at least Python 3.6, I believe
import sys
print(sys.version)

3.7.3 (default, Mar 27 2019, 22:11:17) 
[GCC 7.3.0]


In [3]:
# Requires at least Pandas 0.24.1
import pandas as pd
print(pd.__version__)

0.24.2


In [ ]:
# Display all columns - may take up a lot of memory
pd.options.display.max_columns = None
pd.options.display.float_format = '{:,.2f}'.format
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))   # A lot of memory?

In [4]:
# Hard-coded path name: Change for different machines.
csl_df = pd.read_sas('/home/mbopf/MFMDatasets/CSL_StudyItems/CSLDatasets/csllinkedbypreg.sas7bdat', format = 'sas7bdat', encoding='iso-8859-1')
csl_df.head()

,Sitenum,MomID,Marital,Education,Insurance,Momage,Parity,Gravidity,Hxlivebirth,Hxmacrosomia,...,Fulldilattime,Placenttime,Inducttime,Ripentime,ROMtime,Epiduraltime,Intraantibtime,USdays,LMPdays,onepregflag
0,41,41-00001,1.0,3.0,1.0,34.0,4.0,5.0,4.0,1.0,...,15.0,4.0,361.0,NaN,300.0,343.0,346.0,103.0,271.0,2.0
1,41,41-00001,1.0,3.0,1.0,36.0,5.0,6.0,5.0,2.0,...,18.0,3.0,369.0,NaN,251.0,314.0,NaN,128.0,278.0,2.0
2,41,41-00002,1.0,3.0,1.0,37.0,1.0,3.0,1.0,1.0,...,NaN,1.0,NaN,NaN,1.0,4.0,NaN,128.0,268.0,1.0
3,41,41-00003,1.0,3.0,1.0,36.0,1.0,4.0,1.0,1.0,...,69.0,4.0,382.0,NaN,369.0,364.0,NaN,133.0,273.0,1.0
4,41,41-00004,1.0,2.0,1.0,34.0,2.0,6.0,2.0,1.0,...,12.0,5.0,339.0,NaN,105.0,116.0,455.0,204.0,274.0,1.0


In [ ]:
csl_df.describe(include='all')

In [ ]:
csl_df.agg(['min','max'])

In [5]:
csl_df.dtypes

Sitenum            object
MomID              object
Marital           float64
Education         float64
Insurance         float64
Momage            float64
Parity            float64
Gravidity         float64
Hxlivebirth       float64
Hxmacrosomia      float64
Hxpreterm         float64
Hxstillbirth      float64
HxnumCS           float64
Hxcsection        float64
Preweight         float64
PreBMI            float64
Diabetes          float64
ChronicHBP        float64
Hxheartdis        float64
Hxrenaldis        float64
HxGIdis           float64
Hxdepression      float64
Hxseizure         float64
Hxthyroid         float64
Hxasthma          float64
Hxanemia          float64
HIV               float64
HxSTD             float64
Numfetus          float64
OGTT1h            float64
                   ...   
Deliverytime1     float64
DeliveryYear2     float64
DeliveryHour2     float64
DeliveryDay2       object
Deliverytime2     float64
DeliveryYear3     float64
DeliveryHour3     float64
DeliveryDay3

In [ ]:
# Convert any non-float fields to IntegerArray (Int)
# Note than IntegerArrays are an experimental addition in Pandas 0.24. They
# allow integer columns to contain NaN fields like float columns.
#
# This is a rather brute-force technique that loops through every column
# and every row. There's got to be a more efficient way to do it since it 
# takes a long time and uses up a lot of memory.
def convert_integer (df):
    for col in df.columns:
        intcol_flag = True
        if df[col].dtype == 'float64':   # Assuming dtype is "float64"
            # TODO: Need to remove inner loop - SLOW!
            for val in df[col]:
                # If not NaN and the int() value is different from
                # the float value, then we have an actual float.
                if pd.notnull(val) and abs(val - int(val)) > 1e-6:
                    intcol_flag = False
                    break;
            # If not a float, change it to an Int based on size
            if intcol_flag:
                if df[col].abs().max() < 127:
                    df[col] = df[col].astype('Int8')
                elif df[col].abs().max() < 32767:
                    df[col] = df[col].astype('Int16')
                else:   # assuming no ints greater than 2147483647 
                    df[col] = df[col].astype('Int32') 
#        print(f"{col} is {df[col].dtype}")
    return df

In [ ]:
%%time
df = convert_integer(csl_df)
# CPU times: user 3min, sys: 2min 14s, total: 5min 14s
# Wall time: 5min 14s

In [ ]:
# Convert any non-float fields to IntegerArray (Int)
# Note than IntegerArrays are an experimental addition in Pandas 0.24. They
# allow integer columns to contain NaN fields like float columns.
#
# This is a rather brute-force technique that loops through every column
# and every row. There's got to be a more efficient way to do it since it 
# takes a long time and uses up a lot of memory.
def convert_to_integer (df):
    for col in df.columns:
        intcol_flag = True
        if df[col].dtype == 'float64':   # Assuming dtype is "float64"
            # If not NaN and the int() value is different from
            # the float value, then we have an actual float.
            # s = df[col].apply(lambda x: pd.notnull(x) and abs(x - int(x)) > 1e-6)
            s = df[col].notnull() & df[col].sub(df[col].round()).abs().gt(1e-6)
            if s.any():
                intcol_flag = False
            # If not a float, change it to an Int based on size
            if intcol_flag:
                if df[col].abs().max() < 127:
                    df[col] = df[col].astype('Int8')
                elif df[col].abs().max() < 32767:
                    df[col] = df[col].astype('Int16')
                else:   # assuming no ints greater than 2147483647 
                    df[col] = df[col].astype('Int32') 
#        print(f"{col} is {df[col].dtype}")
    return df

In [ ]:
%%time
df = convert_to_integer(csl_df)

In [ ]:
csl_df.dtypes

In [ ]:
csl_df.shape

In [ ]:
csl_df.info()

In [ ]:
csl_df.describe(include='all')

In [ ]:
csl_df.nunique()

In [ ]:
sites = csl_df.Sitenum
sites.value_counts().plot(kind='bar')

In [ ]:
csl_df['Sitenum'].head()

In [ ]:
csl_df.to_csv('/home/mbopf/MFMDatasets/CSL_StudyItems/mbopf/csl_all.csv')

In [ ]:
cols = csl_df.columns
print(cols)
[x for x in cols]

In [ ]:
csl_df['Sitenum']

In [ ]:
# Clear some memory
import gc
del csl_df
gc.collect()

In [4]:
df = csl_df

In [6]:
import numpy as np
np.random.seed(10)
df = pd.DataFrame(np.random.choice([1, 2, 3.3, 5000, 111111, np.NaN], (3,9)), 
                  columns=[f'col{i}' for i in range(9)])
df['col9'] = ['a','b','c']
df

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9
0,2.0,NaN,111111.0,1.0,2.0,5000.0,111111.0,2.0,NaN,a
1,1.0,NaN,2.0,3.3,1.0,2.0,1.0,3.3,1.0,b
2,111111.0,5000.0,1.0,111111.0,5000.0,1.0,5000.0,3.3,2.0,c


In [11]:
df.dtypes == 'object'

col0    False
col1    False
col2    False
col3    False
col4    False
col5    False
col6    False
col7    False
col8    False
col9     True
dtype: bool

In [20]:
s = pd.cut(df.max(), bins=[0, 127, 32767, 2147483647], labels=['Int8', 'Int16', 'Int32'])
print(s)
s = s.where((df.dtypes=='float') & (df.isnull() | (df%1 == 0)).all())
            # Cast previously       # If all values are 
            # float columns         # "I"nteger-like
print(s)

TypeError: cannot compare a dtyped [object] array with a scalar of type [bool]

In [11]:
for idx, gp in s.groupby(s):
    df.loc[:, gp.index] = df.loc[:, gp.index].astype(idx)

In [15]:
df.dtypes

col0      Int32
col1      Int16
col2      Int32
col3    float64
col4      Int16
col5      Int16
col6      Int32
col7    float64
col8       Int8
dtype: object